# Counter-Factual Regret Minimization

CFRM is an algorithm for developing a **Nash-Equilibrium** in imperfect-information games.  At a high-level, CFRM works by playing against itself and iterativly updating its strategy until it converges on a best strategy.

#### Nash Equilibrium
A Nash Equilibrium strategy is a strategy which is **unexploitable** in the long run.  This means that in the worst-case we will tie our opponent.  In a simple game like tic-tac-toe, this will likely lead to a tie, but poker is so complex that no human player plays a nash-equilibrium strategy so we will be able to profit.  It's important to note that this works by calculating a perfect strategy and does not involve exploiting weaknesses in the opponents play.

Unlike in perfect information games like chess, in imperfect-information games like poker, the true state of the game is not known because we cannot see the other player cards.  Because of this, CFRM does not work on game states and instead works on **Information Sets**

#### Information Sets
An **Information set** is a unique strategic situation that encompasses all known information.  In our case this is...
 - Our two cards
 - The cards on the table
 - The betting history (has the other just checked, what did he do on the flop, ect.)
 
At each information set we will calculate a strategy relating to the frequency we will take an available action.  For example...
 - Bet \$50: 10%
 - Bet \$100: 80%
 - Check: 10%
 
Our **current strategy** for an information set is usually stored in a list of **regrets**

#### Regrets
A regret is a value representing how benificial an action was.  An action with *high-positive* regret should be take over an action an with *negative regret*.  We update our regrets (our current strategy) using an algorithm known as **regret-matching**.

#### Regret Matching

Regret matching works by first calculating the utility of every possible action. For example, if our opponent has throw *ROCK*, our action utilities would be as follows

[ PAPER = 1, ROCK = 0, SCISSORS = -1 ]

Next, we calculate the utility of the action we chose, lets say *PAPER*.  Since PAPER loses to ROCK, our utility would be -1.

Now, we're ready to update our strategy.  To do this we add to each regret the difference between our action utility and the utility of the corresponding action.  What this does, is say for each action "How much better off if I'd taken this action instead of the one I did?".  If the answer is better, it adds regret to that action so it will be taken more often in the future.  If the answer is worse, it subtracts regret.

### Regret Matching Example: Rock, Paper, Scissors

To demonstrate how regret matching-works, we'll use rock, paper, scissors as an example.

In [361]:
import numpy as np
import copy

# We have three actions to choose from
ROCK, SCISSORS, PAPER = 0, 1, 2
ACTIONS = [ROCK, SCISSORS, PAPER]
# ROCK > SCISSORS, SCISSORS > PAPER, PAPER > ROCK

def get_utility(my_action, opp_action):
    if my_action == opp_action: # tie
        return 0
    if (my_action+1)%3 == opp_action: # we win!
        return 1
    else: # lose
        return -1

# Let's set our initial strategy to always throw PAPER
regrets = [0, 1, 0]
# And set our avg_strategy to zeros
strategy_sum = np.zeros(3)

opp_strategy = [0.5, 0.0, 0.5]

# Get strategy through regret matching
def get_strategy(regrets):
    strategy = np.zeros(len(regrets))
    norm_sum = 0
    for i in range(len(regrets)):
        if regrets[i] > 0: # only add positive regrets to norm sum
            strategy[i] = regrets[i]
            norm_sum += strategy[i]
    for i in range(len(regrets)):
        if norm_sum > 0: # calculate normalized strategy
            strategy[i] /= norm_sum
        else:
            strategy[i] = 1 / len(regrets)
            
    return strategy

# Get an action based on a normalized strategy
# strategy = [0.6, 0.2, 0.2]
# 60% chance to return 0, 20% chance to return 1 or 2
def get_action(strategy):
    return np.random.choice(ACTIONS, p=strategy)

# Plays a game and updates regrets
def play(): 
    global regrets
    global strategy_sum
    # get my current strategy
    strategy = get_strategy(regrets)
    # select an action based on my strategy
    my_action = get_action(strategy)
    # select an opponent action 
    opp_action = get_action(strategy)
    
    # try replacing with this and seeing how it responds
#     opp_action = get_action(opp_strategy)
    
    # initialize utility of each action to be zero
    utils = np.zeros(len(ACTIONS))
    # get utility of each action
    for a in ACTIONS:
        utils[a] = get_utility(a, opp_action)
    # update regrets through regret matching
    for a in ACTIONS:
        regrets[a] += utils[a] - utils[my_action]
        # increment avg strategy
        strategy_sum[a] += strategy[a]

# Let's train for 10000 iterations and see if our
# strategy converges to a nash equillibrium
for i in range(1000):
    play()
    if i % 100 == 0:
        print(get_strategy(strategy_sum))

[0. 1. 0.]
[0.34996857 0.24128949 0.40874195]
[0.33494994 0.34510389 0.31994617]
[0.36232344 0.32127017 0.3164064 ]
[0.30514789 0.35707226 0.33777984]
[0.33911095 0.29119312 0.36969594]
[0.31003695 0.34676921 0.34319384]
[0.33033803 0.29976849 0.36989348]
[0.30540924 0.34343958 0.35115119]
[0.32132679 0.31043042 0.36824279]


Cool! Hopefully the strategy is getting close to throwing ROCK, PAPER, and SCISSORS 1/3 of the time each.  This turns out to the be Nash Equilibrium strategy.  You can try repacing the opponent strategy with some other fixed strategy and seeing how that effects ours.

#### CFRM Example: Simplfied Poker

To demonstrate how regret matching is used in CFRM, we'll use a simplified version of poker.  To do this we'll restrict both the **state** and the **action** space.  In our simplified poker game, players are dealt one card.  The first player can then decide to BET 1 chip or CHECK.  If the first player has BET 1 chip, the second player can decide to CALL, FOLD, or RAISE 1 chip.  There will be no reraising

#### Step 1: Game Tree

Let's build a game tree to reflect this simplified poker game.

In [362]:
from copy import deepcopy

BET, RAISE, CHECK, CALL, FOLD = 0, 1, 2, 3, 4
ACTIONS = [BET, RAISE, CHECK, CALL, FOLD]

# Represents the state of the game when building the tree
class GameState:
    def __init__(self):
        self.current_player = 0 # 0: first player, 1: second player
        self.pot = 1 # size of the pot in chips
        self.bets_settled = False # has a player called or folded
        self.player_folded = None # which player has folded
        self.player_bets = [0, 0]
        self.player_cards = [0, 0]
        self.hist = ""
        
    def randomize_cards(self):
        self.player_cards[0] = np.random.randint(10)
        self.player_cards[1] = np.random.randint(10)
        while self.player_cards[1] == self.player_cards[0]:
            self.player_cards[1] = np.random.randint(10)
        
    def get_valid_actions(self):
        return list(filter(lambda a: self.is_action_valid(a), ACTIONS))
    
    def get_infoset_key(self):
        return str(self.player_cards[self.current_player]) + self.hist
    
    def is_action_valid(self, action):
        if action == CHECK:
            # if there is money in the pot, you can't check
            # you must call, fold, or raise
            return np.sum(self.player_bets) == 0
        if action == BET:
            # if there is money in the pot, you can't bet
            # you must raise
            return np.sum(self.player_bets) == 0
        if action == CALL:
            # if there is no money in the pot, you can't call
            return np.sum(self.player_bets) != 0
        if action == FOLD:
            # if there is no money in the pot, don't fold.  Check instead
            return np.sum(self.player_bets) != 0
        if action == RAISE:
            # Only the second player can raise,
            # and the first player has to have alrady bet
            return np.sum(self.player_bets) != 0 and self.current_player == 1
        return False
        
    # apply a valid action to the game state
    # and return a new, updated state
    def apply_action(self, action):
        next_state = deepcopy(self)
        if action == CHECK:
            if next_state.current_player == 0:
                next_state.current_player = 1
            else:
                next_state.bets_settled = True
            next_state.hist += "x"
        if action == BET:
            next_state.player_bets[next_state.current_player] += 1
            next_state.current_player = 1 - next_state.current_player
            next_state.hist += "b"
        if action == RAISE:
            next_state.player_bets[next_state.current_player] += 1
            next_state.pot += np.sum(next_state.player_bets)
            next_state.player_bets = [0, 0]
            next_state.player_bets[next_state.current_player] += 1
            next_state.current_player = 1 - next_state.current_player
            next_state.hist += "r"
        if action == FOLD:
            next_state.player_folded = next_state.current_player
            next_state.bets_settled = True
            next_state.hist += 'f'
        if action == CALL:
            next_state.player_bets[next_state.current_player] += 1
            next_state.pot += np.sum(next_state.player_bets)
            next_state.bets_settled = True
            next_state.hist += 'c'
        return next_state

Although, we haven't actually built the game tree, this will be fine for our purposes.  We can traverse game tree by applying actions recursivily.  This is not very efficient and should not be done in production.

#### Step 2. Algorithm

Lets actually build our CFRM algorithm.

In [369]:
# First, we need a table to store our regrets
# We're going to use a str->arr dictionary where
# str represents or information set
# for example 'Abc' means that we hold an Ace and the action went BET .. CALL
# The value of this array will be regrets as we used before
infosets = dict()

def get_or_create_infoset(n_actions, key):
    global infosets
    if not key in infosets:
        infosets[key] = {
            'regrets': np.ones(n_actions) / n_actions,
            'strategy_sum': np.zeros(n_actions)
        }
    return infosets[key]

# Our main function is called cfr
# 
# hist represents the true game state
# for example 'ATxx' means player 1 has an ACE player two has a TEN and it went CHECK .. CHECK
# 
# state represents the current state (or node we're in)
#
# cfr reach is the counterfactual reach probability of reaching the current node
# A reach probability is the probability of reaching a current node
# For example
#    10% chance to be dealt an A
#    * 50% chance of opponent checking with a J
#    * 50% chance of me raising with an A
#    = 2.5% chance of reaching this node
# A counterfactual reach probabililty only accounts for things outside of our control
# This would mean instead of 2.5% we have a 5% of reaching this node
#    10% chance to be dealt an A
#    * 50% of opponent checking with a J
#    = 5% chance of reaching this node
# 
# The function returns the utility of the current node
# it does this by calling cfr recursivily until it reaches a terminal node
# 
def cfr(state, cfr_reach):
    global infosets
    # if this is a terminal node,
    # return the utility for each player
    if state.bets_settled:
        if not state.player_folded == None:
            return (1 if state.player_folded == 1 else -1) * state.pot
            # return [state.pot, -state.pot] if state.player_folded == 1 else [-state.pot, state.pot]
        # Tie
        if state.player_cards[0] == state.player_cards[1]:
            #return [0, 0]
            return 0
        # player 1 wins
        elif state.player_cards[0] > state.player_cards[1]:
            return state.pot
            #return [state.pot, -state.pot]
        # player 2 wins
        else:
            return -state.pot
            #return [-state.pot, state.pot]
        
    player = state.current_player
    # get all valid actions we can take
    actions = state.get_valid_actions()
    # get our infoset & strategy
    infoset = get_or_create_infoset(len(actions), state.get_infoset_key())
    strategy = get_strategy(infoset['regrets'])
    utils = np.zeros(len(actions))
    node_util = 0
    # calculate utils
    for a in range(len(actions)):
        # calculate new reach probabilty
        child_cfr_reach = cfr_reach.copy()
        child_cfr_reach[1 - player] *= strategy[a]
        utils[a] = cfr(state.apply_action(actions[a]), child_cfr_reach)
        # cummulate utility of action * weighted by likelihood of choosing this action
        node_util += strategy[a] * utils[a]
    # update regrets
    for a in range(len(actions)):
        # Probability of us reaching this node * the regret difference
        infoset['regrets'][a] += cfr_reach[player] * (utils[a] - node_util) * (1 if player == 0 else -1)
        # Cummulate regret
        infoset['strategy_sum'][a] += cfr_reach[player] * strategy[a]
    return node_util
        

# Train our strategy
def train():
    for i in range(10000):
        state = GameState()
        state.randomize_cards()
        cfr(state, [1, 1])
        if i % 1000 == 0:
            print(i, end='\r')
        
train()

In [371]:
def action_to_str(action):
    if action == BET: return 'BET'
    if action == CALL: return 'CALL'
    if action == FOLD: return 'FOLD'
    if action == CHECK: return 'CHECK'
    if action == RAISE: return 'RAISE'
    

# Lets view the root action node
state = GameState()
# state = state.apply_action(BET)

actions = state.get_valid_actions()

print([action_to_str(a) for a in actions])
for key in infosets:
    if key[1:] == '':
        print(key, get_strategy(infosets[key]['strategy_sum']))
        print(key, infosets[key]['regrets'])

['BET', 'CHECK']
7 [0.20015026 0.79984974]
7 [15.26250723 -6.10720485]
3 [0.01600477 0.98399523]
3 [-221.60846246    4.55631002]
9 [0.89402326 0.10597674]
9 [  10.35454392 -124.74410039]
4 [0.01432653 0.98567347]
4 [-361.63736909    4.29744351]
5 [0.00163053 0.99836947]
5 [-215.69086083    2.13345121]
2 [8.15034829e-04 9.99184965e-01]
2 [-114.50458546    2.04972141]
0 [0.50091737 0.49908263]
0 [ 5.19452353 28.64189144]
8 [0.30422569 0.69577431]
8 [24.4010326   1.49193866]
1 [0.1702434 0.8297566]
1 [-34.94750324  15.5775208 ]
6 [4.73933649e-04 9.99526066e-01]
6 [-50.71625676   1.5       ]
